# Mosaic & Sedona

> You can combine the usage of [Mosaic](https://databrickslabs.github.io/mosaic/index.html) with other geospatial libraries. In this example we combine it with [Sedona](https://sedona.apache.org).

## Setup

This notebook will run if you have both Mosaic and Sedona installed on your cluster as described below.

### Install Sedona

To install Sedona, follow the [official Sedona instructions](https://sedona.apache.org/1.5.0/setup/databricks/).

E.g. Add the following maven coordinates to a non-photon cluster [[1](https://docs.databricks.com/en/libraries/package-repositories.html)]. This is showing DBR 12.2 LTS.  

```
org.apache.sedona:sedona-spark-shaded-3.0_2.12:1.5.0
org.datasyslab:geotools-wrapper:1.5.0-28.2
```

### Install Mosaic

Download Mosaic JAR to your local machine (e.g. from [here](https://github.com/databrickslabs/mosaic/releases/download/v_0.3.12/mosaic-0.3.12-jar-with-dependencies.jar) for 0.3.12) and then UPLOAD to your cluster [[1](https://docs.databricks.com/en/libraries/cluster-libraries.html#install-a-library-on-a-cluster)]. 

### Notes

* See instructions for `SedonaContext.create(spark)` [[1](https://sedona.apache.org/1.5.0/tutorial/sql/?h=sedonacontext#initiate-sedonacontext)]. 
* And, Sedona identifies that it might have issues if executed on a [Photon](https://www.databricks.com/product/photon) cluster; again this example is showing DBR 12.2 LTS on the Mosaic 0.3 series.

--- 
 __Last Update__ 01 DEC 2023 [Mosaic 0.3.12]

## Setup

> We are installing Mosaic without SQL functions registered (via Scala) and are installing Sedona SQL as normal.

In [0]:
%scala

// -- spark functions
import org.apache.spark.sql.functions._

// -- mosaic functions
import com.databricks.labs.mosaic.functions.MosaicContext
import com.databricks.labs.mosaic.H3
import com.databricks.labs.mosaic.JTS

val mosaicContext = MosaicContext.build(H3, JTS)
import mosaicContext.functions._

// ! don't register SQL functions !
// - this allows sedona to be the main spatial SQL provider
//mosaicContext.register()

// -- sedona functions
import org.apache.sedona.spark.SedonaContext
val sedona = SedonaContext.create(spark)

import org.apache.spark.sql.functions._
import com.databricks.labs.mosaic.functions.MosaicContext
import com.databricks.labs.mosaic.H3
import com.databricks.labs.mosaic.JTS
mosaicContext: com.databricks.labs.mosaic.functions.MosaicContext = com.databricks.labs.mosaic.functions.MosaicContext@64740153
import mosaicContext.functions._
import org.apache.sedona.spark.SedonaContext
sedona: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@15e8a79a

_When we list user functions, we see all the Sedona provided ones._

In [0]:
%sql 
show user functions like 'st_*'

function
hive_metastore.default.st_geohash
st_3ddistance
st_addpoint
st_affine
st_angle
st_areaspheroid
st_asbinary
st_asewkb
st_asewkt
st_asgeojson


## Queries

> Showing how Sedona (registered Spark SQL) and Mosaic (Scala) can co-exist on the same cluster. Not shown here, but the could also be Mosaic Python bindings.

In [0]:
%scala

val df = Seq("POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))").toDF("wkt")
display(df)

wkt
"POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))"


_Here is a Scala call to use the Sedona (Spark SQL) functions using `selectExpr`._

In [0]:
%scala
display(
  df
    .selectExpr("ST_Area(ST_GeomFromText(wkt)) AS sedona_area")
)

sedona_area
550.0


_Here is Scala call to the same Mosaic-provided `ST_Area` function._

In [0]:
%scala
display(
  df
    .select(st_area($"wkt").as("mosaic_area"))
)

mosaic_area
550.0


_Mosaic + Sedona_

> Showing blending Mosaic calls (in Scala) with Sedona (Spark SQL) calls, using `expr`.

In [0]:
%scala
display(
  df
    .select(
      st_area($"wkt").as("mosaic_area"),                    // <- mosaic (scala)
      expr("ST_Area(ST_GeomFromText(wkt)) AS sedona_area"), // <- sedona (spark sql)
      $"wkt"
    )
)

mosaic_area,sedona_area,wkt
550.0,550.0,"POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))"
